In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re
import json

In [2]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [4]:
resultadoAmsa = sf.query_all(
    """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
amsaIngreso = pd.DataFrame(resultadoAmsa["records"])

amsaIngreso = amsaIngreso[
    [
        "expr0",
        "expr2",
        "expr1",
    ]
]
amsaIngreso = amsaIngreso.rename(
    columns={
        "expr0": "Año",
        "expr1": "Cantidad",
        "expr2": "Mes",
    }
)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

amsaIngreso.replace({"Mes": meses}, inplace=True)

amsaIngreso.to_excel("F:/Reportes SF/Zurich/Cantida_AMSA.xlsx", index=False)
amsaIngreso.head(12)

,Año,Mes,Cantidad
0,2023,Enero,4091
1,2023,Febrero,267
2,2023,Marzo,195
3,2023,Mayo,3150
4,2023,Junio,346
5,2023,Julio,125
6,2023,Agosto,1
7,2023,Septiembre,174
8,2023,Octubre,200
9,2022,Noviembre,3


In [5]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false  
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(8060, 8)

In [6]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7746, 5)

In [7]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(8066, 10)

In [8]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

fin empresas y comienzo de rescate de pacientes


In [9]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1880756, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [10]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1500901, 4)

In [11]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1500901, 13)

In [12]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [ ]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [ ]:
# pacientes_empresas22.shape

In [ ]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [ ]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [ ]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [26]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [27]:
Pacientes_Archivo.head(10)

,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,...,direccion,comuna,ciudad,celularg,RELACION,NOMBRE,apellidos,nombres,email,celular
0,340023382,1,4501433417 Ministro Hales,7606619,1,0,7606619,1,7606619-1,1956-08-13,...,GABRIEL D ANUNZIO,LA REINA,SANTIAGO,98219221,AS,Sanchez Barrera Ruth Irene,Sanchez Barrera,Ruth Irene,RUTHSANCHEZ45@GMAIL.COM,998219221.0
1,340023382,1,4501433417 Ministro Hales,7606619,1,2,7379706,3,7379706-3,1957-02-10,...,GABRIEL D ANUNZIO,LA REINA,SANTIAGO,98219221,CO,Ramirez Oscar,Ramirez,Oscar,RUTHSANCHEZ45@GMAIL.COM,998219221.0
2,340023382,1,4501433417 Ministro Hales,7897579,2,0,7897579,2,7897579-2,1958-06-06,...,PJE LA VALLEJA 3633 B VILLA REMODELACION SANTA...,RECOLETA,NaN,974668393,AS,Alvarez Lopez Santiago Esteban,Alvarez Lopez,Santiago Esteban,santiago.alvarezlopez1@gmail.com,974668393.0
3,340023382,1,4501433417 Ministro Hales,7897579,2,2,9934301,K,9934301-K,1962-07-19,...,PJE LA VALLEJA 3633 B VILLA REMODELACION SANTA...,RECOLETA,NaN,974668393,CO,Rosales Porfidia,Rosales,Porfidia,santiago.alvarezlopez1@gmail.com,974668393.0
4,340023382,1,4501433417 Ministro Hales,7944319,0,0,7944319,0,7944319-0,1959-12-27,...,PJ 3 CS 639 PB J M DE VELASCO,SAN FELIPE,SAN FELIPE,4498265 66316,AS,Baez Castro Maria Del Pilar,Baez Castro,Maria Del Pilar,pily.baez@gmail.com,966316386.0
5,340023382,1,4501433417 Ministro Hales,7944319,0,5,8598976,6,8598976-6,1956-09-28,...,PJ 3 CS 639 PB J M DE VELASCO,SAN FELIPE,SAN FELIPE,4498265 66316,CO,Figari Cruz Andres Francisco,Figari Cruz,Andres Francisco,pily.baez@gmail.com,966316386.0
6,340023382,1,4501433417 Ministro Hales,7980881,4,0,7980881,4,7980881-4,1961-04-15,...,HERNAN VIDELA LIRA 187 POBLACION VISTA HERMOSA,COPIAPÓ,NaN,NaN,AS,Loyola Cervantes Maria Raquel,Loyola Cervantes,Maria Raquel,rlocer@gmail.com,982220418.0
7,340023382,1,4501433417 Ministro Hales,7980881,4,1,6615104,2,6615104-2,1960-05-02,...,HERNAN VIDELA LIRA 187 POBLACION VISTA HERMOSA,COPIAPÓ,NaN,NaN,CO,Nuñez Contreras José Hernan,Nuñez Contreras,José Hernan,rlocer@gmail.com,982220418.0
8,340023382,1,4501433417 Ministro Hales,8000083,9,0,8000083,9,8000083-9,1960-01-05,...,CAMINO LA PIRAMIDE 5822 PC EL ALMENDRAL,HUECHURABA,NaN,94496542,AS,Cifuentes Vera Rebeca,Cifuentes Vera,Rebeca,rebecacifuentes_106@hotmail.com,994496542.0
9,340023382,1,4501433417 Ministro Hales,8253557,8,0,8253557,8,8253557-8,1962-09-09,...,LA OPERA 970,MAIPU,SANTIAGO,951000202,AS,Villa Valenzuela Olga,Villa Valenzuela,Olga,olga.villa@benefits.cl,951000202.0


In [28]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [29]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,...,comuna,ciudad,celularg,RELACION,NOMBRE,apellidos,nombres,email,celular,rut
0,340023382,1,4501433417 Ministro Hales,7606619,1,0,7606619,1,7606619-1,1956-08-13,...,LA REINA,SANTIAGO,98219221,AS,Sanchez Barrera Ruth Irene,Sanchez Barrera,Ruth Irene,RUTHSANCHEZ45@GMAIL.COM,998219221.0,76066191
1,340023382,1,4501433417 Ministro Hales,7606619,1,2,7379706,3,7379706-3,1957-02-10,...,LA REINA,SANTIAGO,98219221,CO,Ramirez Oscar,Ramirez,Oscar,RUTHSANCHEZ45@GMAIL.COM,998219221.0,73797063
2,340023382,1,4501433417 Ministro Hales,7897579,2,0,7897579,2,7897579-2,1958-06-06,...,RECOLETA,NaN,974668393,AS,Alvarez Lopez Santiago Esteban,Alvarez Lopez,Santiago Esteban,santiago.alvarezlopez1@gmail.com,974668393.0,78975792
3,340023382,1,4501433417 Ministro Hales,7897579,2,2,9934301,K,9934301-K,1962-07-19,...,RECOLETA,NaN,974668393,CO,Rosales Porfidia,Rosales,Porfidia,santiago.alvarezlopez1@gmail.com,974668393.0,9934301K
4,340023382,1,4501433417 Ministro Hales,7944319,0,0,7944319,0,7944319-0,1959-12-27,...,SAN FELIPE,SAN FELIPE,4498265 66316,AS,Baez Castro Maria Del Pilar,Baez Castro,Maria Del Pilar,pily.baez@gmail.com,966316386.0,79443190


In [30]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5716\1949753364.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,...,comuna,ciudad,celularg,RELACION,NOMBRE,apellidos,nombres,email,celular,rut
0,340023382,1,4501433417 Ministro Hales,7606619,1,0,7606619,1,7606619-1,1956-08-13,...,LA REINA,SANTIAGO,98219221,AS,Sanchez Barrera Ruth Irene,Sanchez Barrera,Ruth Irene,RUTHSANCHEZ45@GMAIL.COM,998219221.0,76066191
1,340023382,1,4501433417 Ministro Hales,7606619,1,2,7379706,3,7379706-3,1957-02-10,...,LA REINA,SANTIAGO,98219221,CO,Ramirez Oscar,Ramirez,Oscar,RUTHSANCHEZ45@GMAIL.COM,998219221.0,73797063
2,340023382,1,4501433417 Ministro Hales,7897579,2,0,7897579,2,7897579-2,1958-06-06,...,RECOLETA,NaN,974668393,AS,Alvarez Lopez Santiago Esteban,Alvarez Lopez,Santiago Esteban,santiago.alvarezlopez1@gmail.com,974668393.0,78975792
3,340023382,1,4501433417 Ministro Hales,7897579,2,2,9934301,K,9934301-K,1962-07-19,...,RECOLETA,NaN,974668393,CO,Rosales Porfidia,Rosales,Porfidia,santiago.alvarezlopez1@gmail.com,974668393.0,9934301K
4,340023382,1,4501433417 Ministro Hales,7944319,0,0,7944319,0,7944319-0,1959-12-27,...,SAN FELIPE,SAN FELIPE,4498265 66316,AS,Baez Castro Maria Del Pilar,Baez Castro,Maria Del Pilar,pily.baez@gmail.com,966316386.0,79443190


In [31]:
def siPrimerCero(y):
    if y == "00":
        return "Nones"
    if y[0] == "0":
        y = y[1:]
    return y

In [32]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))

In [33]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))

Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5716\620742920.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5716\620742920.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [34]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(200, 32)

In [35]:
Pacientes_con_id.head()

,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,nacimiento,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,340023382,1,4501433417 Ministro Hales,7606619,1,0,7606619,1,7606619-1,1956-08-13,...,True,0018c00002axV9LAAU,76066191,Sanchez Barrera Ruth Irene,RUT,None,None,None,Mujer,None
1,340023382,1,4501433417 Ministro Hales,7606619,1,2,7379706,3,7379706-3,1957-02-10,...,True,0018c00002axVA2AAM,73797063,Ramirez Oscar,RUT,None,None,None,Hombre,None
2,340023382,1,4501433417 Ministro Hales,7897579,2,0,7897579,2,7897579-2,1958-06-06,...,True,0018c00002axVBzAAM,78975792,Alvarez Lopez Santiago Esteban,RUT,None,None,None,Hombre,None
3,340023382,1,4501433417 Ministro Hales,7897579,2,2,9934301,K,9934301-K,1962-07-19,...,True,0018c00002axVDyAAM,9934301K,Rosales Porfidia,RUT,None,None,None,Mujer,None
4,340023382,1,4501433417 Ministro Hales,7944319,0,0,7944319,0,7944319-0,1959-12-27,...,True,0018c00002axjljAAA,79443190,Baez Castro Maria Del Pilar,RUT,None,None,None,Mujer,None


In [36]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [37]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Mujer",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5716\2367721946.py:88: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [38]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [39]:
""" print(dia_hoy) """

' print(dia_hoy) '

In [40]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [41]:
campaña_seleccionada.shape

(971162, 13)

In [42]:
""" f campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

' f campaña != "MetLife":\n    pacientes_en_campaña = pd.merge(\n        left=campaña_seleccionada,\n        right=Pacientes_con_id,\n        how="left",\n        left_on="Id Account",\n        right_on="Id Account Paciente",\n    )\n    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"\n    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"\n    pacientes_en_campaña.to_excel(\n        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False\n    )\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False\n    )\n '

In [43]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

' if campaña != "MetLife1":\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    if ("poliza" in campaña_en_pacientes.columns) == True:\n        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)\n\n    campaña_en_pacientes = campaña_en_pacientes.rename(\n        columns={"Programas_por_Empresas__c": "programas por empresa"}\n    )\n    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]\n    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[\n        "programas por empresa"\n    ]\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False\n    ) '

In [44]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [45]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)